In [1]:
import glob
import os
import pickle
from music21 import converter, instrument, note, chord

#############################
# MIDI Processing Utilities #
#############################

def ingest_midi(filename):
    print("Ingesting... %s" % filename)
    midi = converter.parse(filename)
    
    # If there are multiple instrument tracks, focus
    # only on the primary instrument in the MIDI.
    parts = instrument.partitionByInstrument(midi)
    if parts and len(parts) > 0:
        midi_events = parts.parts[0].recurse()
    else:
        midi_events = midi.flat.notes
    
    # Parse MIDI events for Notes and Chords
    parsed_notes = []
    for midi_event in midi_events:
        if isinstance(midi_event, note.Note):
            # For Note, add pitch to sequence
            parsed_notes.append(midi_event.pitch.name)
        elif isinstance(midi_event, chord.Chord):
            # For Chord, join multiple pitches to sequence
            parsed_notes.append(' '.join(pitch.name for pitch in midi_event.pitches))
            
    return parsed_notes, sorted(set(parsed_notes))


def ingest_midis(file_glob):
    notes_by_file = {}
    vocab = set()
    
    for filename in glob.glob(file_glob):
        file_notes, file_vocab = ingest_midi(filename)
        
        notes_by_file[filename] = file_notes
        vocab.update(file_vocab)
        
    return notes_by_file, vocab


def mkdir(filename):
    if not os.path.exists(os.path.dirname(filename)):
        os.makedirs(os.path.dirname(filename))


def save_to_file(content, filename):
    mkdir(filename)
    pickle.dump(content, open(filename, 'wb'))


def load_from_file(filename):
    return pickle.load(open(filename, 'rb'))

In [2]:
############################
# Transformation Utilities #
############################

def generate_note_to_int(vocab):
    return dict((note, number) for number, note in enumerate(vocab))


def generate_int_to_note(vocab):
    return dict((number, note) for number, note in enumerate(vocab))


def generate_sequences(notes_by_file, note_to_int, sequence_length = 100):
    x = []
    y = []

    for notes in notes_by_file.values():
        for i in range(0, len(notes) - sequence_length, 1):
            # Sequence of N Input Notes --> 1 Output Note
            input_sequence = notes[i:i + sequence_length]
            output_sequence = notes[i + sequence_length]

            x.append([note_to_int[c] for c in input_sequence])
            y.append(note_to_int[output_sequence])

    return x, y

In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint

###################
# Model Utilities #
###################

def build_model(input_width, output_width, model_file = None):
    model = Sequential()

    model.add(LSTM(256, input_shape=(input_width, 1), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dropout(0.3))
    model.add(Dense(output_width))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model


def train_model(model, x, y, model_file, epochs=200, batch_size=128):
    checkpoint = ModelCheckpoint(model_file, monitor='loss', save_best_only=True)
    callbacks_list = [checkpoint]
    model.fit(x, y, epochs=epochs, batch_size=batch_size, callbacks=callbacks_list)

/Users/bbamsch/.virtualenvs/cmpe257/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
######################
# Process MIDI Files #
######################

from keras.utils import np_utils
import numpy

def main():
    # Load Cached Notes (if exists) or ingest all MIDI files
    note_cache_file = 'model/cached-notes.pkl'
    if os.path.exists(note_cache_file):
        notes_by_file, vocab = load_from_file(note_cache_file)
    else:
        notes_by_file, vocab = ingest_midis("data/*.mid")
        save_to_file((notes_by_file, vocab), note_cache_file)

    # Generate Sequences of 100 input Notes -> 1 output Note
    sequence_length = 100
    note_to_int = generate_note_to_int(vocab)
    x, y = generate_sequences(
        notes_by_file=notes_by_file,
        note_to_int=note_to_int,
        sequence_length=sequence_length)
    
    num_sequences = len(x)
    num_vocab = len(vocab)
    
    # Reshape & Normalize Input Notes
    x = numpy.reshape(x, (num_sequences, sequence_length, 1)) / num_vocab
    # Transform Output to One Hot Encoding
    y = np_utils.to_categorical(y)

    # Build ML Model
    model = build_model(sequence_length, num_vocab)

    # Run Training w/ Checkpoints
    train_model(
        model=model,
        x=x,
        y=y,
        model_file="model/weights-improvements-{epoch:03d}-{loss:.4f}.hdf5"
    )

main()

Epoch 1/200
 9472/30956 [========>.....................] - ETA: 5:32 - loss: 4.2315

KeyboardInterrupt: 